# 1. Repaso de probabilidad

## 1.1. Probabilidad condicional

### $P(A|B) = \frac{P(A ∩ B)}{P(B)}$

![](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*8GB3SuoGNOaFW5rk8zZp3w.png)
![](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8a/Conditional_probability_venn_1-10.svg/744px-Conditional_probability_venn_1-10.svg.png)

Referencias de esta notebook:
- https://towardsdatascience.com/conditional-probability-with-a-python-example-fd6f5937cd2
- https://www.statology.org/conditional-probability-in-python/
- https://machinelearningmastery.com/bayes-theorem-for-machine-learning/
- https://towardsdatascience.com/bayes-theorem-with-example-for-data-science-professionals-55b4d52f8967
- https://towardsdatascience.com/bayes-rule-with-a-simple-and-practical-example-2bce3d0f4ad0

**Analicemos la siguiente base de datos:** https://www.kaggle.com/datasets/uciml/student-alcohol-consumption e intentemos responder la pregunta:

### 1) ¿Qué probabilidad hay de que un estudiante pase con calificación mayor de 8 dado que faltó 10 veces o más a clase?

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data/StudentAlcoholConsumption/student-mat.csv')
data.head(3)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10


In [3]:
df = data[['G3', 'absences']]
df.head(3)

,G3,absences
0,6,6
1,6,4
2,10,10


G3 - final grade (numeric: from 0 to 20, output target) 

In [4]:
df['calif'] = np.where(df['G3']*5 >= 80, 1, 0)
df.head(3)

/tmp/ipykernel_12867/1835273762.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['calif'] = np.where(df['G3']*5 >= 80, 1, 0)


,G3,absences,calif
0,6,6,0
1,6,4,0
2,10,10,0


absences - number of school absences (numeric: from 0 to 93), considerar un alto ausentismo si faltó 10 veces o más.

In [5]:
df['ausencias_altas'] = np.where(df['absences'] >= 10, 1, 0)
df.head(3)

/tmp/ipykernel_12867/3134837393.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ausencias_altas'] = np.where(df['absences'] >= 10, 1, 0)


,G3,absences,calif,ausencias_altas
0,6,6,0,0
1,6,4,0,0
2,10,10,0,1


In [6]:
df['contador'] = 1
df.head(10)

/tmp/ipykernel_12867/1458621534.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['contador'] = 1


,G3,absences,calif,ausencias_altas,contador
0,6,6,0,0,1
1,6,4,0,0,1
2,10,10,0,1,1
3,15,2,0,0,1
4,10,4,0,0,1
5,15,10,0,1,1
6,11,0,0,0,1
7,6,6,0,0,1
8,19,0,1,0,1
9,15,0,0,0,1


In [7]:
df = df[['calif','ausencias_altas','contador']]
df.head()

,calif,ausencias_altas,contador
0,0,0,1
1,0,0,1
2,0,1,1
3,0,0,1
4,0,0,1


In [8]:
#produce contingency table to summarize raw data
cross_data = pd.crosstab(index=df['calif'], columns=df['ausencias_altas'], margins=True)
cross_data.head()

ausencias_altas,0,1,All
calif,,,
0,277,78,355
1,35,5,40
All,312,83,395


Sea P(A) la probabilidad de obtener calificación mayor a 80.
Sea P(B) la probabilidad de faltar a 10 o más clases.

P(A|B) es la probabilidad de obtener una calificación mayor a 80 dado que faltó a 10 o más clases.

In [9]:
cross_data.iloc[2,2]

395

In [10]:
total = cross_data.iloc[2,2] # 277+35+78+5
A = cross_data.iloc[1,2] # 35+5
B = cross_data.iloc[2,1] #78+5

In [11]:
PA = A/total
PB = B/total
PA, PB

(0.10126582278481013, 0.21012658227848102)

Para $P(A ∩ B)$ tenemos:

In [12]:
AintB = cross_data.iloc[1,1]
PAintB = AintB/total
PAintB

0.012658227848101266

Calculemos $P(A|B) = \frac{P(A ∩ B)}{P(B)}$

In [13]:
PAdadoB = PAintB/PB

print("La probabilidad de que un estudiante obtenga una calificación mayor"
      "al 80% si falta a 10 o más clases es: {:.3f}".format(PAdadoB))

print("De cada 100 estudiantes, solo {:.3f}% obtendría una calificación mayor al 80%".format(PAdadoB*100))

La probabilidad de que un estudiante obtenga una calificación mayoral 80% si falta a 10 o más clases es: 0.060
De cada 100 estudiantes, solo 6.024% obtendría una calificación mayor al 80%


### 2. Ejercicio: Calcular la probabilidad de que un estudiante obtenga una calificación menor a 8 si consume alcohol entre semana de manera alta (Dalc >= 4).

## 1.2. Probabilidad total

### $P(B) = P(B ∩ A_1 ) + P(B ∩ A_2 ) + P(B ∩ A_3 ) +...+ P(B ∩ A_n)$
    

### 3. Práctica: Usando los datos del caso anterior, probar que $P(B) = P(B ∩ A_1 ) + P(B ∩ A_2)$, considérese que P(B) es la probabilidad total de que un estudiante obtenga una calificación mayor igual al 80%. ¿Quiénes serían $A_1$ y $A_2$?

## 1.3. Teorema de Bayes

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*CnoTGGO7XeUpUMeXDrIfvA.png)

In [14]:
# calculate P(A|B) given P(A), P(B|A), P(B|not A)
def bayes_theorem(p_a, p_b_given_a, p_b_given_not_a):
    # calculate P(not A)
    not_a = 1 - p_a
    # calculate P(B)
    p_b = p_b_given_a * p_a + p_b_given_not_a * not_a
    # calculate P(A|B)
    p_a_given_b = (p_b_given_a * p_a) / p_b
    return p_a_given_b

Calculemos $P(A|not B) = \frac{P(A ∩ not B)}{P(not B)}$

In [15]:
cross_data.head()

ausencias_altas,0,1,All
calif,,,
0,277,78,355
1,35,5,40
All,312,83,395


### Ejemplo 1 (práctica). Calculemos P(B|A) del caso anterior.

In [19]:
notB = cross_data.iloc[2,0]
PnotB = notB/total
AintnotB = cross_data.iloc[1,0]
PAintnotB =  AintnotB/total
PAdadoNOB = PAintnotB/PnotB

result = bayes_theorem(PB, PAdadoB, PAdadoNOB)
# summarize
print("P(B|A) = {:.3f} equivale a la probabilidad de que un estudiante haya faltado más de diez veces"
      "dado que obtuvo una calificación superior al 80%".format(result))

P(B|A) = 0.125 equivale a la probabilidad de que un estudiante haya faltado más de diez vecesdado que obtuvo una calificación superior al 80%


### Ejemplo 2. Un detector de spam identifica que el 20% del spam de un usuario contiene la palabra "free". Si se asume que el 0.1% de los correos no marcados como spam incluyen la palabra "free" y que el 50% de todos los correos recibidos por el usuario son spam, ¿cuál sería la probabilidad de que un mensaje sea spam si contiene la palabra "free"?



    P(Free | Spam) = 0.20

    P(Free | Non Spam) = 0.001

    P(Spam) = 0.50 => P(Non Spam) = 0.50

    P(Spam | Free) = ?
    
    
![](https://miro.medium.com/v2/resize:fit:720/format:webp/0*ZZmHng0wEjr2xyYW.png)

### Ejercicio, ¿cuál sería la probabilidad de que un mensaje sea spam si contiene la palabra "free" si se asume que solo el 30% de los correos recibidos por el usuario son spam?